In [1]:
# Dependencies
from splinter import Browser
from bs4 import BeautifulSoup
import re
import requests
import pandas as pd

## NASA Mars News

In [2]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
news_url = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"
browser.visit(news_url)

In [4]:
latest_news_title = browser.find_by_tag("div[class='content_title']").first.text
latest_news_p = browser.find_by_tag("div[class='article_teaser_body']").first.text
latest_news_link_html = browser.find_by_tag("div[class='image_and_description_container']").html
news_soup = BeautifulSoup(latest_news_link_html, "html.parser")
latest_news_url = "https://mars.nasa.gov" + news_soup.a["href"]

In [5]:
print(latest_news_title)
print(latest_news_p)
print(latest_news_url)

Space Samples Link NASA's Apollo 11 and Mars 2020
While separated by half a century, NASA's Apollo 11 and Mars 2020 missions share the same historic goal: returning samples to Earth.
https://mars.nasa.gov/news/8502/space-samples-link-nasas-apollo-11-and-mars-2020/


## JPL Mars Space Images - Featured Image

In [6]:
jpl_url= "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(jpl_url)

In [7]:
latest_mars_image_html = browser.find_by_tag("li[class='slide']").html
jpl_soup = BeautifulSoup(latest_mars_image_html, "html.parser")
latest_mars_image_url = "https://www.jpl.nasa.gov" + jpl_soup.a["data-fancybox-href"]

In [8]:
print(latest_mars_image_url)

https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA23384_hires.jpg


## Mars Weather

In [9]:
mars_weather_url = "https://twitter.com/marswxreport?lang=en"
browser.visit(mars_weather_url)

In [10]:
mars_weather_html = browser.find_by_tag("p[class='TweetTextSize TweetTextSize--normal js-tweet-text tweet-text']").html
weather_soup_text = BeautifulSoup(mars_weather_html, "html.parser").text
weather_first_split = ("S" + weather_soup_text.split("InSight s")[1])
mars_weather_string = weather_first_split.split("hPa")[0] + "hPa"
print(mars_weather_string)

Sol 251 (2019-08-11) low -101.0ºC (-149.7ºF) high -26.5ºC (-15.8ºF)
winds from the SSE at 4.1 m/s (9.2 mph) gusting to 17.5 m/s (39.1 mph)
pressure at 7.60 hPa


## Mars Facts

In [11]:
mars_facts_url = "https://space-facts.com/mars/"
tables = pd.read_html(mars_facts_url)

In [37]:
mars_facts_df = tables[1]
mars_facts_df.columns = ["Description", "Value"]
mars_facts_df = mars_facts_df.set_index("Description")
mars_facts_html = mars_facts_df.to_html()

In [38]:
mars_facts_df 

,Value
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


## Mars Hemispheres




In [21]:
mars_hemispheres_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(mars_hemispheres_url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

hemisphere_img_urls = []

hemisphere_names = ["Cerberus", "Schiaparelli", "Syrtis", "Valles"]
ind = 0

for i in range(4):
    
    browser.click_link_by_partial_text(hemisphere_names[ind])
    html = browser.html
    soup = BeautifulSoup(html, "html.parser")
    title_text = browser.find_by_tag("h2[class='title']").text.split(" Enhanced")[0]
    img_text = str(soup.find_all("img", class_="wide-image")[0])
    img_url = 'https://astrogeology.usgs.gov' + img_text.split('src="')[1].split('"/>')[0]
    
    title_link_dict = {"title": title_text, "img_url": img_url}
    hemisphere_img_urls.append(title_link_dict)
    
    browser.visit(mars_hemispheres_url)
    ind+=1   

In [22]:
hemisphere_img_urls

[{'title': 'Cerberus Hemisphere',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}]

In [24]:
mars_data = {"news_title": latest_news_title, 
             "news_p": latest_news_p,
             "news_url": latest_news_url,
             "featured_image_url": latest_mars_image_url,
             "mars_weather": mars_weather_string,
             "mars_facts_table": mars_facts_html,
             "hemisphere_image_urls": hemisphere_img_urls
                 
    }